In [1]:
"""
This code is meant to be ran on Shrek (or Donkey). If labelling frames is necessary, those frames
will be moved to a different computer for labelling, then moved back to Shrek (or Donkey) using "Label-videos.ipynb"

This code can run deeplabcut from scratch (start new project, label frames, train network, analyze videos) or can 
start at different points in the pipeline

"""
import fnmatch
import pandas as pd
import shutil
import os
import sys
from ruamel.yaml import YAML
import yaml
os.environ["DLClight"]="True"
import deeplabcut
import glob

import subprocess

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarning: Passing (type, 1) or 

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [6]:
"""
Set desired processes and variables

is_new_project will create a new project folder and configuration file
"""
is_new_project = True
label_frames = False
make_training_data = True
train_model = True
analyze_videos = True
filter_videos = True
create_labeled_video = True

# set project dir, project, and experimenter. Get video list. Note: videos need to be in folder called /videos/ in project directory
project_directory = '/home/jts3256/DLC_videos'
project = 'Han_202012'
experimenter = 'Joe'

videos = glob.glob(project_directory + '/videos/*.avi')

# set body part list and other frame label variables
body_part_list = ['shoulder','elbow1','elbow2','wrist1','wrist2','hand1','hand2','hand3','pointX','pointY','pointZ']
num_frames_per_vid = 20 # default = 20
training_frac = [0.95] # default = [0.95], needs to be a list

# determine which model to train/use (set as empty '' if training a new model)
# this is currently not implemented

model_loc = ''


In [7]:
if(is_new_project): # create project
    linux_config_file = deeplabcut.create_new_project(project, experimenter, videos,
                                                 working_directory=project_directory,copy_videos=True)
    # overwrite project_directory
    project_directory = os.path.dirname(os.path.abspath(linux_config_file))
    
    with open(linux_config_file) as file:
        data = yaml.load(file, Loader = yaml.FullLoader)

        data['bodyparts'] = body_part_list
        data['TrainingFraction'] = training_frac
        data['numframes2pick'] = num_frames_per_vid # default = 20
        
        # update video list
        temp = data['video_sets']
        videos = list(temp.keys())
        
    with open(linux_config_file,'w') as file:
        yaml.dump(data,file)
    
else: # project is already created, check to make sure it exists then continue
    if(not os.path.isdir(project_directory)):
        print("Video project folder does not exist, check please")
        exit() # stop code
    linux_config_file = project_directory + '/config.yaml'

print(videos)

Created "/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/videos"
Created "/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/labeled-data"
Created "/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/training-datasets"
Created "/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/dlc-models"
Copying the videos
/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/videos/Han_20201204_00003_trimmed.avi
/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/videos/Han_20201204_00002_trimmed.avi
/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/videos/Han_20201204_00008_trimmed.avi
/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/videos/Han_20201204_00006_trimmed.avi
/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/videos/Han_20201204_00007_trimmed.avi
/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/videos/Han_20201204_00001_trimmed.avi
/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/videos/Han_20201204_00005_trimmed.avi
/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/videos/Han_202

In [6]:
"""
if we are labeling frames: 
    extract frames, then move frames to a second computer. Label frames on second computer using Label-videos.ipynb
    Then move videos back to this computer and continue
"""
if(label_frames):
    deeplabcut.extract_frames(linux_config_file,mode='automatic',userfeedback=False)
    # run Label-videos on second computer
      

In [9]:
"""
Make a new training dataset if requested
"""
if(make_training_data):
    deeplabcut.create_training_dataset(linux_config_file,userfeedback=False,windows2linux=True)

/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/training-datasets/iteration-0/UnaugmentedDataSet_Han_202012Dec14  already exists!
Annotation data converted to unix format...
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [ ]:
if(train_model):
    deeplabcut.train_network(linux_config_file)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10]],
 'all_joints_names': ['shoulder',
                      'elbow1',
                      'elbow2',
                      'wrist1',
                      'wrist2',
                      'hand1',
                      'hand2',
                      'hand3',
                      'pointX',
                      'pointY',
                      'pointZ'],
 'batch_size': 1,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Han_202012Dec14/Han_202012_Joe95shuffle1.mat',
 'dataset_type': 'default',
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/jts3256/.local/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref

Selecting single-animal trainer
Starting with imgaug pose-dataset loader (=default).
Batch Size is 1
Initializing ResNet
Loading ImageNet-pretrained resnet_50
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/jts3256/DLC_videos/Han_202012-Joe-2020-12-14/dlc-models/iteration-0/Han_202012Dec14-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'default', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise

iteration: 1000 loss: 0.0244 lr: 0.005
iteration: 2000 loss: 0.0143 lr: 0.005


In [ ]:
deeplabcut.analyze_videos(linux_config_file,videos, save_as_csv=True)
deeplabcut.filterpredictions(linux_config_file,videos,filtertype='spline')
deeplabcut.plot_trajectories(linux_config_file,videos)
deeplabcut.create_labeled_video(linux_config_file,videos,filtered=True)